In [13]:
import unittest
from collections import namedtuple
from itertools import combinations
from typing import Sequence, Hashable, TypeVar, Generic, Container, Tuple

Action = TypeVar('Action', bound=Hashable)
Shape = namedtuple('Shape',['sides', 'shade', 'texture'])
State = TypeVar('State', bound=Hashable)
Likelihood = TypeVar('Likelihood', float, int)

# Test PCFG programs

In [14]:
# TODO: The logic in this function has not been checked
def two(*args, state) -> bool:
    '''Check the conditions for two shapes in the given state based on arguments.'''

    # Two of a feature anywhere
    if len(args) == 1 and args[0] in ['square', 'circle', 'triangle', 'low', 'medium', 'high', 'present', 'not_present']:
        feature = args[0]
        count = 0
        for shape in (state.shape1, state.shape2, state.shape3):
            if (feature in shape.sides or feature in shape.shade or feature in shape.texture):
                count += 1
        return count == 2
    
    # Compare two shapes for given number of features
    elif (len(args) == 2) and (args[0] in ['2', '3']) and args[1] in ['same','unique']:
        feature_count, comparison = args
        feature_count = int(feature_count)
        # Check to see if exactly two shapes share the specified number of features, and no more
        if comparison == 'same':
            matching_pairs = 0
            for shape_combo in combinations([state.shape1, state.shape2, state.shape3], 2):
                features1 = {shape_combo[0].sides, shape_combo[0].shade, shape_combo[0].texture}
                features2 = {shape_combo[1].sides, shape_combo[1].shade, shape_combo[1].texture}
                common_features = len(features1.intersection(features2))
                # Check if the number of common features matches the specified count
                if common_features == feature_count:
                    matching_pairs += 1
            # Return True only if exactly one pair matches the criteria
            return matching_pairs == 1
        else:
            return False

    # TODO: Check logic here
    # Compare two shapes anywhere over all features
    elif len(args) == 1 and args[0] in ['same', 'unique']:
        feature = args[0]
        features = [shape.sides for shape in (state.shape1, state.shape2, state.shape3)]
        features += [shape.shade for shape in (state.shape1, state.shape2, state.shape3)]
        features += [shape.texture for shape in (state.shape1, state.shape2, state.shape3)]
        unique_features = set(features)
        if feature == 'same':
            return len(unique_features) == 1
        elif feature == 'unique':
            return len(features) == len(unique_features)
    
    # TODO: Check logic here
    # Compare two locations for one feature
    elif len(args) == 2 and args[1] in ['(0,1)', '(0,2)', '(1,2)']:
        feature, location = args
        location = int(location)
        shape = (state.shape1, state.shape2, state.shape3)[location]
        return (feature in shape.sides or feature in shape.shade or feature in shape.texture)

    # TODO: Check logic here
    # Compare two locations across all features
    elif len(args) == 2 and args[1] in ['0', '1', '2']:
        pass

    # TODO: Add cases for comparing multiple features

    else:
        raise ValueError('Invalid call for the two shapes rule.')


In [15]:
# tests for the two function

def test_two():

    # Test case 1: Two of a feature anywhere
    shape1 = Shape(sides='square', shade='low', texture='present')
    shape2 = Shape(sides='circle', shade='medium', texture='not_present')
    shape3 = Shape(sides='triangle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    assert two('square', state) == True
    assert two('circle', state) == True
    assert two('triangle', state) == True
    assert two('low', state) == True
    assert two('medium', state) == True
    assert two('high', state) == True
    assert two('present', state) == True
    assert two('not_present', state) == True

    # Test case 1: Two shapes have two features in common. 
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='square', shade='low', texture='not_present')
    shape3 = Shape(sides='square', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == True
    except AssertionError as e:
        raise AssertionError("Test case 1 failed: " + str(e))

    # Test case 2: All shapes have different number of sides
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='not_present')
    shape3 = Shape(sides='circle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 2 failed: " + str(e))

    # Test case 3: Two shapes have the same number of sides
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='not_present')
    shape3 = Shape(sides='square', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 3 failed: " + str(e))

    # Test case 4: Two shapes have different number of sides
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='not_present')
    shape3 = Shape(sides='circle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 4 failed: " + str(e))

    # Test case 5: Two shapes have the same shade
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='medium', texture='not_present')
    shape3 = Shape(sides='circle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == True
    except AssertionError as e:
        raise AssertionError("Test case 5 failed: " + str(e))

    # Test case 6: Two shapes have different shade
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='not_present')
    shape3 = Shape(sides='circle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 6 failed: " + str(e))

    # Test case 7: Two shapes have the same texture
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='not_present')
    shape3 = Shape(sides='circle', shade='high', texture='not_present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 7 failed: " + str(e))

    # Test case 8: Two shapes have different texture
    shape1 = Shape(sides='square', shade='medium', texture='not_present')
    shape2 = Shape(sides='triangle', shade='low', texture='present')
    shape3 = Shape(sides='circle', shade='high', texture='present')
    state = State(shape1, shape2, shape3)
    try:
        assert two('2', 'same', state=state) == False
    except AssertionError as e:
        raise AssertionError("Test case 6 failed: " + str(e))

    print("All test cases passed!")

test_two()

TypeError: 'TypeVar' object is not callable